In [ ]:
# importation de bibliothèques:
import tensorflow as tf
from tensorflow.keras.datasets import mnist, cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, LSTM, BatchNormalization, MaxPooling2D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.metrics import Accuracy, Precision, Recall, AUC
import matplotlib.pyplot as plt
import time
import pickle

# Installation du drive
from google.colab import drive
drive.mount('/content/drive')
# Spécifier le chemin du dossier de destination dans Google Drive
base_path = '/content/drive/My Drive/initiation_recherche/final/'

# Fonctions personnelles de sauvegarde de resultats

def save_with_pickle(obj, filename):
    with open(filename, 'wb') as file:
        pickle.dump(obj, file)

# Fonction pour charger les données depuis un fichier pickle
def load_pkl(filename):
    with open(filename, 'rb') as file:
        return pickle.load(file)

Mounted at /content/drive


In [ ]:


# Télécharger les ensembles de données MNIST et CIFAR-10
(mnist_train_images, mnist_train_labels), (mnist_test_images, mnist_test_labels) = mnist.load_data()
(cifar_train_images, cifar_train_labels), (cifar_test_images, cifar_test_labels) = cifar10.load_data()

# Normaliser les images pour MNIST et CIFAR-10
mnist_train_images = mnist_train_images.astype('float32') / 255.0
mnist_test_images = mnist_test_images.astype('float32') / 255.0
cifar_train_images = cifar_train_images.astype('float32') / 255.0
cifar_test_images = cifar_test_images.astype('float32') / 255.0

# Redimensionner les images MNIST pour le modèle CNN (ajout d'un canal)
mnist_train_images_cnn = mnist_train_images.reshape((-1, 28, 28, 1))
mnist_test_images_cnn = mnist_test_images.reshape((-1, 28, 28, 1))

# Redimensionner les images CIFAR-10 pour le modèle CNN
cifar_train_images_cnn = cifar_train_images.reshape((-1, 32, 32, 3))
cifar_test_images_cnn = cifar_test_images.reshape((-1, 32, 32, 3))

# Redimensionner les images CIFAR-10 pour le modèle RNN
cifar_train_images_rnn = cifar_train_images.reshape((-1, 32, 32 * 3))  # Redimensionner en (32, 96)
cifar_test_images_rnn = cifar_test_images.reshape((-1, 32, 32 * 3))

# One-hot encoding des étiquettes pour MNIST et CIFAR-10
mnist_train_labels = to_categorical(mnist_train_labels, 10)
mnist_test_labels = to_categorical(mnist_test_labels, 10)
cifar_train_labels = to_categorical(cifar_train_labels, 10)
cifar_test_labels = to_categorical(cifar_test_labels, 10)


170498071/170498071 [==============================] - 11s 0us/step


## modèles CNN

In [ ]:
# modèle CNN pour mnist :
model_cnn_mnist = Sequential([
    Conv2D(256, (3, 3), activation='relu', input_shape=(28, 28, 1)),  # Adapté pour CIFAR-10
    BatchNormalization(),
    MaxPooling2D(2, 2),
    Conv2D(256, (3, 3), activation='relu'),
    BatchNormalization(),
    Conv2D(256, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(128, activation='relu'),
    BatchNormalization(),
    Dense(64, activation='relu'),
    BatchNormalization(),
    Dense(10, activation='softmax')  # 10 classes pour CIFAR-10
])


# Modèle CNN pour CIFAR-10
model_cnn_cifar = Sequential([
    Conv2D(256, (3, 3), activation='relu', input_shape=(32, 32, 3)),  # Adapté pour CIFAR-10
    BatchNormalization(),
    MaxPooling2D(2, 2),
    Conv2D(256, (3, 3), activation='relu'),
    BatchNormalization(),
    Conv2D(256, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(128, activation='relu'),
    BatchNormalization(),
    Dense(64, activation='relu'),
    BatchNormalization(),
    Dense(10, activation='softmax')  # 10 classes pour CIFAR-10
])





## modèles RNN

In [ ]:
# modèle RNN :
model_rnn_mnist = Sequential([
    LSTM(512, return_sequences=True, input_shape=(28, 28)),  # 32 lignes de 32x3 pixels
    BatchNormalization(),
    LSTM(256),  # 256 neurones
    BatchNormalization(),
    Dense(256, activation='relu'),  # 256 neurones
    BatchNormalization(),
    Dense(128, activation='relu'),  # 128 neurones
    BatchNormalization(),
    Dense(10, activation='softmax')  # 10 classes pour CIFAR-10
])

# Modèle RNN pour CIFAR-10
model_rnn_cifar = Sequential([
    LSTM(512, return_sequences=True, input_shape=(32, 96)),  # 32 lignes de 32x3 pixels
    BatchNormalization(),
    LSTM(256),  # 256 neurones
    BatchNormalization(),
    Dense(256, activation='relu'),  # 256 neurones
    BatchNormalization(),
    Dense(128, activation='relu'),  # 128 neurones
    BatchNormalization(),
    Dense(10, activation='softmax')  # 10 classes pour CIFAR-10
])


## modèles DNN

In [ ]:
# modèle DNN
model_dnn_mnist = Sequential([
    Flatten(input_shape=(28,28)),
    Dense(512, activation='relu'),  # 512 neurones
    BatchNormalization(),
    Dense(512, activation='relu'),  # 512 neurones
    BatchNormalization(),
    Dense(512, activation='relu'),  # 512 neurones
    BatchNormalization(),
    Dense(512, activation='relu'),  # 512 neurones
    BatchNormalization(),
    Dense(512, activation='relu'),  # 512 neurones
    BatchNormalization(),
    Dense(256, activation='relu'),  # 256 neurones
    BatchNormalization(),
    Dense(256, activation='relu'),  # 256 neurones
    BatchNormalization(),
    Dense(128, activation='relu'),  # 128 neurones
    BatchNormalization(),
    Dense(128, activation='relu'),  # 128 neurones
    BatchNormalization(),
    Dense(128, activation='relu'),   # 128 neurones
    BatchNormalization(),
    Dense(64, activation='relu'),   # 64 neurones
    BatchNormalization(),
    Dense(10, activation='softmax')  # 10 classes pour CIFAR-10
])

# Modèle DNN pour CIFAR-10
model_dnn_cifar = Sequential([
    Flatten(input_shape=(32, 32, 3)),  # Adapté pour CIFAR-10
    Dense(512, activation='relu'),  # 512 neurones
    BatchNormalization(),
    Dense(512, activation='relu'),  # 512 neurones
    BatchNormalization(),
    Dense(256, activation='relu'),  # 256 neurones
    BatchNormalization(),
    Dense(256, activation='relu'),  # 256 neurones
    BatchNormalization(),
    Dense(128, activation='relu'),  # 128 neurones
    BatchNormalization(),
    Dense(128, activation='relu'),  # 128 neurones
    BatchNormalization(),
    Dense(128, activation='relu'),   # 128 neurones
    BatchNormalization(),
    Dense(64, activation='relu'),   # 64 neurones
    BatchNormalization(),
    Dense(10, activation='softmax')  # 10 classes pour CIFAR-10
])



In [ ]:
# compiler modèles :

models_mnist = [model_cnn_mnist, model_rnn_mnist, model_dnn_mnist]
for model in models_mnist:
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])


models_cifar = [model_cnn_cifar, model_rnn_cifar, model_dnn_cifar]
for model in models_cifar:
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])


In [ ]:
# MNIST

# Pour le modèle CNN mnist
print("Résumé du modèle CNN mnist:")
model_cnn_mnist.summary()

# Pour le modèle RNN mnist
print("\nRésumé du modèle RNN mnist:")
model_rnn_mnist.summary()

# Pour le modèle DNN
print("\nRésumé du modèle DNN mnist:")
model_dnn_mnist.summary()


Résumé du modèle CNN mnist:
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 256)       2560      
                                                                 
 batch_normalization (Batch  (None, 26, 26, 256)       1024      
 Normalization)                                                  
                                                                 
 max_pooling2d (MaxPooling2  (None, 13, 13, 256)       0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 11, 256)       590080    
                                                                 
 batch_normalization_1 (Bat  (None, 11, 11, 256)       1024      
 chNormalization)                                                
                            

In [ ]:
# CIFAR :

# Pour le modèle CNN
print("Résumé du modèle CNN cifar:")
model_cnn_cifar.summary()

# Pour le modèle RNN
print("\nRésumé du modèle RNN cifar:")
model_rnn_cifar.summary()

# Pour le modèle DNN
print("\nRésumé du modèle DNN cifar:")
model_dnn_cifar.summary()


Résumé du modèle CNN cifar:
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 30, 30, 256)       7168      
                                                                 
 batch_normalization_5 (Bat  (None, 30, 30, 256)       1024      
 chNormalization)                                                
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 15, 15, 256)       0         
 g2D)                                                            
                                                                 
 conv2d_4 (Conv2D)           (None, 13, 13, 256)       590080    
                                                                 
 batch_normalization_6 (Bat  (None, 13, 13, 256)       1024      
 chNormalization)                                                
                          

In [ ]:
# entrainement CNN MNIST :


start_time = time.time()


history_cnn_mnist = model_cnn_mnist.fit(mnist_train_images_cnn,
                                        mnist_train_labels,
                                        epochs=10,
                                        batch_size=64,
                                        validation_data=(mnist_test_images_cnn, mnist_test_labels))

training_time_cnn_mnist = time.time() - start_time

print(f'Temps d\'entraînement : {training_time_cnn_mnist:.2f} secondes')

# Pour le modèle CNN MNIST


save_with_pickle(history_cnn_mnist, base_path + 'history_cnn_mnist_accuracy.pkl')


Epoch 1/10
938/938 [==============================] - 15s 9ms/step - loss: 0.0925 - accuracy: 0.9729 - val_loss: 0.0391 - val_accuracy: 0.9873
Epoch 2/10
938/938 [==============================] - 8s 9ms/step - loss: 0.0400 - accuracy: 0.9877 - val_loss: 0.0321 - val_accuracy: 0.9889
Epoch 3/10
938/938 [==============================] - 8s 9ms/step - loss: 0.0300 - accuracy: 0.9909 - val_loss: 0.0281 - val_accuracy: 0.9905
Epoch 4/10
938/938 [==============================] - 8s 9ms/step - loss: 0.0246 - accuracy: 0.9921 - val_loss: 0.0237 - val_accuracy: 0.9921
Epoch 5/10
938/938 [==============================] - 8s 9ms/step - loss: 0.0203 - accuracy: 0.9936 - val_loss: 0.2220 - val_accuracy: 0.9552
Epoch 6/10
938/938 [==============================] - 8s 9ms/step - loss: 0.0156 - accuracy: 0.9953 - val_loss: 0.0322 - val_accuracy: 0.9898
Epoch 7/10
938/938 [==============================] - 9s 9ms/step - loss: 0.0147 - accuracy: 0.9955 - val_loss: 0.0227 - val_accuracy: 0.9930
Epoch

In [ ]:
# Entraînement du modèle CNN pour CIFAR-10
start_time = time.time()

history_cnn_cifar = model_cnn_cifar.fit(cifar_train_images_cnn,
                                        cifar_train_labels,
                                        epochs=10,
                                        batch_size=64,
                                        validation_data=(cifar_test_images_cnn, cifar_test_labels)  # Données de validation pour CIFAR-10
)

training_time_cnn_cifar = time.time() - start_time
print(f'Temps d\'entraînement : {training_time_cnn_cifar:.2f} secondes')

# Pour le modèle CNN CIFAR-10

save_with_pickle(history_cnn_cifar, base_path + 'history_cnn_cifar_accuracy.pkl')


Epoch 1/10
782/782 [==============================] - 12s 11ms/step - loss: 1.2956 - accuracy: 0.5394 - val_loss: 1.9294 - val_accuracy: 0.3391
Epoch 2/10
782/782 [==============================] - 8s 10ms/step - loss: 0.8637 - accuracy: 0.6986 - val_loss: 1.2636 - val_accuracy: 0.5667
Epoch 3/10
782/782 [==============================] - 8s 10ms/step - loss: 0.6630 - accuracy: 0.7684 - val_loss: 0.9220 - val_accuracy: 0.6858
Epoch 4/10
782/782 [==============================] - 8s 10ms/step - loss: 0.5084 - accuracy: 0.8234 - val_loss: 0.9238 - val_accuracy: 0.6996
Epoch 5/10
782/782 [==============================] - 8s 10ms/step - loss: 0.3749 - accuracy: 0.8718 - val_loss: 0.9047 - val_accuracy: 0.7136
Epoch 6/10
782/782 [==============================] - 8s 10ms/step - loss: 0.2591 - accuracy: 0.9116 - val_loss: 1.1503 - val_accuracy: 0.6715
Epoch 7/10
782/782 [==============================] - 8s 10ms/step - loss: 0.1911 - accuracy: 0.9345 - val_loss: 1.1612 - val_accuracy: 0.704

In [ ]:
# Graphes CNN MNIST

# Tracer la perte
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history_cnn_mnist.history['loss'], label='Train Loss')
plt.plot(history_cnn_mnist.history['val_loss'], label='Validation Loss')
plt.title('Loss Over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

# Tracer la précision
plt.subplot(1, 2, 2)
plt.plot(history_cnn_mnist.history['accuracy'], label='Train Accuracy')
plt.plot(history_cnn_mnist.history['val_accuracy'], label='Validation Accuracy')
plt.title('Accuracy Over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.show()

In [ ]:
# Graphes CNN cifar

# Tracer la perte
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history_cnn_cifar.history['loss'], label='Train Loss')
plt.plot(history_cnn_cifar.history['val_loss'], label='Validation Loss')
plt.title('Loss Over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

# Tracer la précision
plt.subplot(1, 2, 2)
plt.plot(history_cnn_cifar.history['accuracy'], label='Train Accuracy')
plt.plot(history_cnn_cifar.history['val_accuracy'], label='Validation Accuracy')
plt.title('Accuracy Over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.show()

In [ ]:
# entrainement RNN mnist

start_time = time.time()


history_rnn_mnist = model_rnn_mnist.fit(mnist_train_images,
                            mnist_train_labels,
                            epochs=10,
                            batch_size=64,
                            validation_data=(mnist_test_images, mnist_test_labels))

training_time_rnn_mnist = time.time() - start_time

print(f'Temps d\'entraînement : {training_time_rnn_mnist:.2f} secondes')

# sauvegarder le résultat

save_with_pickle(history_rnn_mnist, base_path + 'history_rnn_mnist_accuracy.pkl')



Epoch 1/10
938/938 [==============================] - 16s 11ms/step - loss: 0.2855 - accuracy: 0.9077 - val_loss: 0.1379 - val_accuracy: 0.9584
Epoch 2/10
938/938 [==============================] - 10s 11ms/step - loss: 0.1062 - accuracy: 0.9675 - val_loss: 0.1056 - val_accuracy: 0.9682
Epoch 3/10
938/938 [==============================] - 10s 11ms/step - loss: 0.0745 - accuracy: 0.9775 - val_loss: 0.0840 - val_accuracy: 0.9762
Epoch 4/10
938/938 [==============================] - 10s 11ms/step - loss: 0.0630 - accuracy: 0.9811 - val_loss: 0.0801 - val_accuracy: 0.9767
Epoch 5/10
938/938 [==============================] - 10s 10ms/step - loss: 0.0515 - accuracy: 0.9838 - val_loss: 0.0532 - val_accuracy: 0.9826
Epoch 6/10
938/938 [==============================] - 10s 11ms/step - loss: 0.0446 - accuracy: 0.9864 - val_loss: 0.0567 - val_accuracy: 0.9833
Epoch 7/10
938/938 [==============================] - 10s 11ms/step - loss: 0.0421 - accuracy: 0.9868 - val_loss: 0.0446 - val_accuracy:

In [ ]:
# Entraînement du modèle RNN pour CIFAR-10
start_time = time.time()

history_rnn_cifar = model_rnn_cifar.fit(cifar_train_images_rnn,
                                        cifar_train_labels,
                                        epochs=10,
                                        batch_size=64,
                                        validation_data=(cifar_test_images_rnn, cifar_test_labels))

training_time_rnn_cifar = time.time() - start_time
print(f'Temps d\'entraînement : {training_time_rnn_cifar:.2f} secondes')

# sauvegarder le résultat

save_with_pickle(history_rnn_cifar, base_path + 'history_rnn_cifar_accuracy.pkl')


Epoch 1/10
782/782 [==============================] - 15s 13ms/step - loss: 1.7904 - accuracy: 0.3534 - val_loss: 3.0948 - val_accuracy: 0.1417
Epoch 2/10
782/782 [==============================] - 9s 12ms/step - loss: 1.4804 - accuracy: 0.4644 - val_loss: 1.5547 - val_accuracy: 0.4225
Epoch 3/10
782/782 [==============================] - 9s 12ms/step - loss: 1.3276 - accuracy: 0.5215 - val_loss: 1.4512 - val_accuracy: 0.4826
Epoch 4/10
782/782 [==============================] - 9s 12ms/step - loss: 1.2197 - accuracy: 0.5597 - val_loss: 1.4030 - val_accuracy: 0.4998
Epoch 5/10
782/782 [==============================] - 9s 12ms/step - loss: 1.1285 - accuracy: 0.5949 - val_loss: 1.2410 - val_accuracy: 0.5610
Epoch 6/10
782/782 [==============================] - 9s 12ms/step - loss: 1.0526 - accuracy: 0.6253 - val_loss: 1.3607 - val_accuracy: 0.5319
Epoch 7/10
782/782 [==============================] - 9s 11ms/step - loss: 0.9726 - accuracy: 0.6512 - val_loss: 1.2077 - val_accuracy: 0.578

In [ ]:
# Graphes RNN mnist
# Tracer la perte
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history_rnn_mnist.history['loss'], label='Train Loss')
plt.plot(history_rnn_mnist.history['val_loss'], label='Validation Loss')
plt.title('Loss Over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

# Tracer la précision
plt.subplot(1, 2, 2)
plt.plot(history_rnn_mnist.history['accuracy'], label='Train Accuracy')
plt.plot(history_rnn_mnist.history['val_accuracy'], label='Validation Accuracy')
plt.title('Accuracy Over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.show()

In [ ]:
# Graphes RNN cifar

# Tracer la perte
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history_rnn_cifar.history['loss'], label='Train Loss')
plt.plot(history_rnn_cifar.history['val_loss'], label='Validation Loss')
plt.title('Loss Over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

# Tracer la précision
plt.subplot(1, 2, 2)
plt.plot(history_rnn_cifar.history['accuracy'], label='Train Accuracy')
plt.plot(history_rnn_cifar.history['val_accuracy'], label='Validation Accuracy')
plt.title('Accuracy Over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.show()

In [ ]:
# entrainement DNN mnist

start_time = time.time()


history_dnn_mnist = model_dnn_mnist.fit(mnist_train_images,
                            mnist_train_labels,
                            epochs=10,
                            batch_size=64,
                            validation_data=(mnist_test_images, mnist_test_labels))

training_time_dnn_mnist = time.time() - start_time

print(f'Temps d\'entraînement : {training_time_dnn_mnist:.2f} secondes')

# sauvegarder le résultat

save_with_pickle(history_dnn_mnist, base_path + 'history_dnn_mnist_accuracy.pkl')


Epoch 1/10
938/938 [==============================] - 19s 13ms/step - loss: 0.3243 - accuracy: 0.9031 - val_loss: 0.1843 - val_accuracy: 0.9514
Epoch 2/10
938/938 [==============================] - 12s 13ms/step - loss: 0.1682 - accuracy: 0.9512 - val_loss: 0.1257 - val_accuracy: 0.9642
Epoch 3/10
938/938 [==============================] - 12s 13ms/step - loss: 0.1399 - accuracy: 0.9592 - val_loss: 0.1281 - val_accuracy: 0.9611
Epoch 4/10
938/938 [==============================] - 12s 13ms/step - loss: 0.1260 - accuracy: 0.9643 - val_loss: 0.0958 - val_accuracy: 0.9702
Epoch 5/10
938/938 [==============================] - 12s 13ms/step - loss: 0.1069 - accuracy: 0.9695 - val_loss: 0.0806 - val_accuracy: 0.9757
Epoch 6/10
938/938 [==============================] - 12s 13ms/step - loss: 0.0992 - accuracy: 0.9718 - val_loss: 0.1051 - val_accuracy: 0.9707
Epoch 7/10
938/938 [==============================] - 12s 13ms/step - loss: 0.0895 - accuracy: 0.9748 - val_loss: 0.0851 - val_accuracy:

In [ ]:
# Entraînement du modèle DNN pour CIFAR-10

start_time = time.time()

history_dnn_cifar = model_dnn_cifar.fit(cifar_train_images,
                                        cifar_train_labels,
                                        epochs=10,
                                        batch_size=64,
                                        validation_data=(cifar_test_images, cifar_test_labels)
)

training_time_dnn_cifar = time.time() - start_time
print(f'Temps d\'entraînement : {training_time_dnn_cifar:.2f} secondes')

# sauvegarder le résultat

save_with_pickle(history_dnn_cifar, base_path + 'history_dnn_cifar_accuracy.pkl')



Epoch 1/10
782/782 [==============================] - 14s 11ms/step - loss: 1.8365 - accuracy: 0.3440 - val_loss: 1.8035 - val_accuracy: 0.3569
Epoch 2/10
782/782 [==============================] - 8s 10ms/step - loss: 1.6357 - accuracy: 0.4136 - val_loss: 1.7311 - val_accuracy: 0.3691
Epoch 3/10
782/782 [==============================] - 8s 10ms/step - loss: 1.5452 - accuracy: 0.4480 - val_loss: 1.8852 - val_accuracy: 0.3357
Epoch 4/10
782/782 [==============================] - 8s 10ms/step - loss: 1.4859 - accuracy: 0.4689 - val_loss: 1.8627 - val_accuracy: 0.3670
Epoch 5/10
782/782 [==============================] - 8s 10ms/step - loss: 1.4324 - accuracy: 0.4904 - val_loss: 1.6841 - val_accuracy: 0.4086
Epoch 6/10
782/782 [==============================] - 8s 10ms/step - loss: 1.3807 - accuracy: 0.5094 - val_loss: 1.4352 - val_accuracy: 0.4887
Epoch 7/10
782/782 [==============================] - 8s 10ms/step - loss: 1.3343 - accuracy: 0.5234 - val_loss: 1.5524 - val_accuracy: 0.452

In [ ]:
# Graphes DNN mnist

# Tracer la perte
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history_dnn_mnist.history['loss'], label='Train Loss')
plt.plot(history_dnn_mnist.history['val_loss'], label='Validation Loss')
plt.title('Loss Over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

# Tracer la précision
plt.subplot(1, 2, 2)
plt.plot(history_dnn_mnist.history['accuracy'], label='Train Accuracy')
plt.plot(history_dnn_mnist.history['val_accuracy'], label='Validation Accuracy')
plt.title('Accuracy Over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.show()

In [ ]:
# Graphes DNN cifar

# Tracer la perte
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history_dnn_cifar.history['loss'], label='Train Loss')
plt.plot(history_dnn_cifar.history['val_loss'], label='Validation Loss')
plt.title('Loss Over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

# Tracer la précision
plt.subplot(1, 2, 2)
plt.plot(history_dnn_cifar.history['accuracy'], label='Train Accuracy')
plt.plot(history_dnn_cifar.history['val_accuracy'], label='Validation Accuracy')
plt.title('Accuracy Over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.show()

In [ ]:
# Rapports de classification

# fonction d'impression d e rapport de classification :
from sklearn.metrics import classification_report
import numpy as np

def print_classification_report(model, X_test, y_test, model_name):
    predictions = model.predict(X_test)
    predictions = np.argmax(predictions, axis=1)
    y_test_argmax = np.argmax(y_test, axis=1) # Si les étiquettes sont en one-hot encoding
    report = classification_report(y_test_argmax, predictions)
    print(f"Rapport de classification pour {model_name}:\n")
    print(report)
    print("\n" + "-"*80 + "\n")


# Exemple d'appel de la fonction pour chaque modèle
print_classification_report(model_cnn_mnist, mnist_test_images_cnn, mnist_test_labels, "CNN sur MNIST")
print_classification_report(model_cnn_cifar, cifar_test_images_cnn, cifar_test_labels, "CNN sur CIFAR-10")
print_classification_report(model_rnn_mnist, mnist_test_images_rnn, mnist_test_labels, "RNN sur MNIST")
print_classification_report(model_rnn_cifar, cifar_test_images_rnn, cifar_test_labels, "RNN sur CIFAR-10")
print_classification_report(model_dnn_mnist, mnist_test_images_dnn, mnist_test_labels, "DNN sur MNIST")
print_classification_report(model_dnn_cifar, cifar_test_images_dnn, cifar_test_labels, "DNN sur CIFAR-10")


# Comparatifs :

In [ ]:
# comparatif :


import matplotlib.pyplot as plt

# Chargement des historiques pour chaque modèle

# history_cnn = load_pkl('history_cnn.pkl')
# history_rnn = load_pkl('history_rnn.pkl')
# history_dnn = load_pkl('history_dnn.pkl')

# Fonction pour tracer les courbes de perte et de précision
def plot_history(histories, title):
    plt.figure(figsize=(14, 5))

    # Tracé des pertes
    plt.subplot(1, 2, 1)
    for name, history in histories:
        plt.plot(history['loss'], label=f'{name} Train')
        plt.plot(history['val_loss'], label=f'{name} Validation')
    plt.title('Loss - ' + title)
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    # Tracé des précisions
    plt.subplot(1, 2, 2)
    for name, history in histories:
        plt.plot(history['accuracy'], label=f'{name} Train')
        plt.plot(history['val_accuracy'], label=f'{name} Validation')
    plt.title('Accuracy - ' + title)
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.show()

# Comparaison des modèles
plot_history([('CNN', history_cnn), ('RNN', history_rnn), ('DNN', history_dnn)], 'Model Comparison')


In [ ]:
# matrice de confusion

from sklearn.metrics import confusion_matrix
import seaborn as sns

def plot_confusion_matrix(model, X_test, y_test, title):
    predictions = model.predict(X_test)
    predictions = np.argmax(predictions, axis=1)
    y_test_argmax = np.argmax(y_test, axis=1) # Si les étiquettes sont en one-hot encoding
    cm = confusion_matrix(y_test_argmax, predictions)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt="d")
    plt.title('Confusion Matrix - ' + title)
    plt.ylabel('Actual Label')
    plt.xlabel('Predicted Label')
    plt.show()

# Exemple d'utilisation
# plot_confusion_matrix(model_cnn, mnist_test_images_cnn, mnist_test_labels, 'CNN')


In [ ]:
# rapport de classification
from sklearn.metrics import classification_report

def print_classification_report(model, X_test, y_test):
    predictions = model.predict(X_test)
    predictions = np.argmax(predictions, axis=1)
    y_test_argmax = np.argmax(y_test, axis=1) # Si les étiquettes sont en one-hot encoding
    report = classification_report(y_test_argmax, predictions)
    print(report)

# Exemple d'utilisation
# print_classification_report(model_cnn, mnist_test_images_cnn, mnist_test_labels)


In [ ]:
# courbe roc et auc :
from sklearn.metrics import roc_curve, auc
from tensorflow.keras.utils import to_categorical

def plot_roc_curve(model, X_test, y_test, num_classes):
    # Prédictions et One-hot encoding si nécessaire
    y_pred = model.predict(X_test)
    y_test_cat = to_categorical(y_test, num_classes)

    # Calcul de la courbe ROC pour chaque classe
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    for i in range(num_classes):
        fpr[i], tpr[i], _ = roc_curve(y_test_cat[:, i], y_pred[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])

    # Tracer toutes les courbes ROC
    plt.figure(figsize=(12, 8))
    for i in range(num_classes):
        plt.plot(fpr[i], tpr[i], label=f'Class {i} (area = {roc_auc[i]:0.2f})')
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve')
    plt.legend(loc="lower right")
    plt.show()

# Exemple d'utilisation pour une classification multi-classes
# plot_roc_curve(model_cnn, mnist_test_images_cnn, mnist_test_labels, num_classes=10)
